In [ ]:
import torch
import torch.nn as nn

import random
import math
import numpy as np
import matplotlib.pyplot as plt
import pytorch_lighthing as pl

In [ ]:
#Vocab


NOTE_SIZE = 128
DUR_SIZE = 160
TIM_SIZE = 1000
VEL_SIZE = 128


NOTE_TOKS = [f'n{i}' for i in range(NOTE_SIZE)] 
DUR_TOKS = [f'd{i}' for i in range(DUR_SIZE)]
TIM_TOKS = [f't{i}' for i in range(TIM_SIZE)]
VEL_TOKS = [f'v{i}' for i in range(VEL_SIZE)]

BOS_TOK = "BOS"
# Le token dummy sert seulement à initialiser les mots du vocab à partir de l'index 1, conformément aux prérequis de la fonction vocab()
VOCAB = ["dummy"] + [BOS_TOK] + NOTE_TOKS + DUR_TOKS + TIM_TOKS + VEL_TOKS 

DICT = [(element, index) for index, element in enumerate(VOCAB)]


In [ ]:
from torchtext.vocab import vocab
from collections import OrderedDict

custom_vocab = vocab(OrderedDict(DICT))
itos_vocab = custom_vocab.get_itos()

In [ ]:
import numpy as np
from tqdm import tqdm
from music21 import *
import os


# Load the MIDI file
midi_file = midi.MidiFile()

les_tokens = []

# Dossier contenant les fichiers MIDI pour l'entraînement
folder_path = "train_data"  

# Get all the file names in the folder
file_names = os.listdir(folder_path)
for f in tqdm(file_names):
    print(f)
    midi_file = midi.MidiFile()
    midi_file.open(folder_path + "/" +f)
    midi_file.read()
    midi_file.close()
    # Create a stream from the MIDI file
    stream = midi.translate.midiFileToStream(midi_file)

    # Iterate over the notes in the stream and extract the note information
    last_time = 0

    for note in stream.flat.notes:
        if note.isNote:
            note_pitch = note.pitch.midi
            # A terme il faudra arrondir plutot que de prendre la partie entiere
            note_duration = int(note.duration.quarterLength*4)
            note_offset = int(note.offset*4 - last_time)
            last_time = note.offset*4
            note_velocity = note.volume.velocity
            les_tokens.append(NOTE_TOKS[note_pitch])
            les_tokens.append(DUR_TOKS[note_duration])
            les_tokens.append(TIM_TOKS[note_offset])
            les_tokens.append(VEL_TOKS[note_velocity])

        if note.isChord:

            for note2 in note:
                note_pitch = note2.pitch.midi
                note_duration = int(note.duration.quarterLength*4)
                note_offset = int(note.offset*4 - last_time)
                last_time = note.offset*4
                note_velocity = note2.volume.velocity
                les_tokens.append(NOTE_TOKS[note_pitch])
                les_tokens.append(DUR_TOKS[note_duration])
                les_tokens.append(TIM_TOKS[note_offset])
                les_tokens.append(VEL_TOKS[note_velocity])
